In [1]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

In [5]:
np.full(10, 1)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [21]:
n = 300
m = 3*n
eps = 0.1
data_dim = 10
mu = 0
mu_out = 5
par_mu = np.full(data_dim, mu)
par_sd = np.eye(data_dim)
out_mu = np.full(data_dim, mu_out)
out_sd = np.eye(data_dim)
par_reg1 = 0.1
par_reg2 = 0.001
true_alpha = [par_mu, par_sd]

exper_iter = 1 #デバッグ後変える
optim_iter = 500
L = 100
learn_par = 1
dicay_par = 0.5

def sigmoid(x):
    return 1/(np.exp(-x) + 1)

def deriv_sigmoid(x):
    return sigmoid(x)*(1-sigmoid(x))

def g_up(t, s):
    return sigmoid(s) + deriv_sigmoid(s)*(t-s) + (t-s)**2/20

def g_lo(t, s):
    return sigmoid(s) + deriv_sigmoid(s)*(t-s) - (t-s)**2/20

In [22]:
data = np.random.multivariate_normal(mean= par_mu, cov = par_sd, size = int(n*(1-eps)))
contamination = np.random.multivariate_normal(mean = out_mu, cov = out_sd, size = (n - int(n*(1-eps))))
data = np.concatenate([data, contamination])

In [ ]:
res = [0 for i in range(exper_iter)]
for i in range(exper_iter):
    print("%d/%d" %(i+1, exper_iter))
    data = np.random.multivariate_normal(loc = par_mu, scale = par_sd, size = int(n*(1-eps)))
    contamination = np.random.multivariate_normal(loc = out_mu, scale = out_sd, size = (n - int(n*(1-eps))))
    data = np.concatenate([data, contamination])
    np.random.shuffle(data)
    alpha_hist = []
    alpha = [np.mean(data, axis=0), np.cov(data, rowvar=0)]
    for j in tqdm(range(1, optim_iter+1)):
        z = np.random.normal(mean=alpha[0], cov=alpha[1], size = m)

        def major_func(par, past_par):
            new_beta = par[0:2*data_dim+1]; new_b = par[2*data_dim+1]; beta = past_par[0:2*data_dim+1]; b = past_par[2*data_dim+1]
            A = np.mean(g_lo(np.dot(np.stack([z, z**2], axis=1).reshape(m, 2*data_dim),new_beta) - new_b, np.dot(np.stack([z, z**2].reshape(m, 2*data_dim), axis=1),beta) - b))
            B = np.mean(g_lo(np.dot(np.stack([data, data**2], axis=1).reshape(n, 2*data_dim),new_beta) - new_b, np.dot(np.stack([data, data**2], axis=1).reshape(n, 2*data_dim),beta) - b))
            reg = par_reg1*np.sum(np.sqrt(new_beta**2))
            return -(A-B) + reg

        l = 0; par = np.random.normal(scale = 0.1, size = 2*data_dim + 1)
        while(l<L):
            op = minimize(major_func, x0 = np.zeros(2*data_dim +1), args = par)
            par = op.x
            l+=1
        # ここまで多次元
        alpha_m = alpha[0]; alpha_v = alpha[1]
        mgrad = np.stack([(z-alpha_m)/alpha_v, ((z-alpha_m)**2-alpha_v)/(2*alpha_v)], axis = 1)
        sig_ = sigmoid(np.dot(np.stack([z, z**2], axis=1),par[0:2])- par[2])[:,np.newaxis]

        tmp_alpha = alpha*(1-learn_par*par_reg2) - learn_par/j**dicay_par * np.mean(mgrad*sig_, axis = 0)
        discount = 0.5; cnt = 1
        while tmp_alpha[1] < 0:
            tmp_alpha = alpha - learn_par/j**dicay_par * discount**cnt * np.mean(mgrad*sig_, axis = 0)
            cnt +=1
        
        alpha = tmp_alpha
        alpha_hist.append(alpha)
    res[i] = alpha_hist

